In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets, transforms, models
import numpy as np
from matplotlib import pyplot as plt
import sys
import os
sys.path.append(os.path.abspath('..'))
from base_trainer import Trainer
from model import SimpleCNN
import json

In [2]:
NUM_CLASSES = 10           # CIFAR-10
# Загружаем CIFAR-10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_dataset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
dataloader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

class_indices = {class_idx: [] for class_idx in range(NUM_CLASSES)}
for idx, (_, label) in enumerate(train_dataset):
    class_indices[label].append(idx)


EPOCHS = 20
learning_rate = 5e-4

val_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [4]:
tmp = models.resnet50().cuda()

In [5]:
print(len(dataloader.dataset))

50000


In [6]:
all_confidences = []
model = SimpleCNN(NUM_CLASSES).to(DEVICE)
with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(DEVICE)  # Переносим данные на устройство модели
            outputs = model(inputs)
            # outputs = tmp(inputs)
            probabilities = torch.softmax(outputs, dim=1)
            confidence_scores, _ = torch.max(probabilities, dim=1)  # Максимальная вероятность для каждого примера
            all_confidences.append(confidence_scores)

        # Конкатенируем все уверенности в один тензор
        confidence_scores = torch.cat(all_confidences)

In [12]:
val_dict = {}
for p in [0.01, 0.1, 0.2]:
    # Разделяем данные
    initial_indices = []
    pool_indices = []
    # for class_idx, indices in class_indices.items():
    #     np.random.shuffle(indices)  # Перемешиваем индексы внутри класса
    #     split_idx = int(len(indices) * p)  # Вычисляем границу разделения
    #     initial_indices.extend(indices[:split_idx])
    
    model = SimpleCNN(NUM_CLASSES).to(DEVICE)
    
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate )
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

        
    # Находим индексы образцов с наименьшей уверенностью
    least_confident_indices = torch.argsort(1 - confidence_scores)[:int(len(dataloader.dataset) * p)]
    print(len(least_confident_indices))
    samples = [dataloader.dataset[i] for i in least_confident_indices]
        

    train_dataloader = DataLoader(samples, batch_size=128, shuffle=True)

    trainer = Trainer(model = model,
                      optimizer=optimizer,
                      pool_loader=None,
                      criterion=criterion ,
                      train_loader=train_dataloader,
                      val_loader = val_dataloader,
                      device=DEVICE,
                      scheduler=scheduler)
    trainer.fit(EPOCHS)
    f1 = max(trainer.val_f1)
    del optimizer 
    del criterion
    del scheduler
    del model
    del trainer
    val_dict[p] = f1
    torch.cuda.empty_cache()
    print(f"f1 for random init: p = {p}, f1 = {val_dict[p]}")

500


Validating: 100%|██████████| 79/79 [00:01<00:00, 39.52it/s]


Validation Loss: 2.2231, Accuracy: 0.1795, F1 Score: 0.1143
Epoch 1/20 - Train Loss: 2.2751, Val Loss: 2.2231, Acc: 0.1795, F1: 0.1143


Validating: 100%|██████████| 79/79 [00:01<00:00, 40.85it/s]


Validation Loss: 2.1341, Accuracy: 0.2456, F1 Score: 0.1971
Epoch 2/20 - Train Loss: 2.1245, Val Loss: 2.1341, Acc: 0.2456, F1: 0.1971


...

Validation Loss: 1.0897, Accuracy: 0.6192, F1 Score: 0.6171
Epoch 18/20 - Train Loss: 0.7623, Val Loss: 1.0897, Acc: 0.6192, F1: 0.6171


Validating: 100%|██████████| 79/79 [00:01<00:00, 41.56it/s]


Validation Loss: 1.0876, Accuracy: 0.6252, F1 Score: 0.6205
Epoch 19/20 - Train Loss: 0.7244, Val Loss: 1.0876, Acc: 0.6252, F1: 0.6205


Validating: 100%|██████████| 79/79 [00:01<00:00, 41.36it/s]

Validation Loss: 1.0751, Accuracy: 0.6342, F1 Score: 0.6311
Epoch 20/20 - Train Loss: 0.7049, Val Loss: 1.0751, Acc: 0.6342, F1: 0.6311
f1 for random init: p = 0.2, f1 = 0.631071272373181


{0.01: 0.3457578408268053, 0.1: 0.5709083333098373, 0.2: 0.6295679530845513}
